# Processamento da Fonte de Dados

Este notebook realiza o processo de ETL (extract, transform, load) para as fontes de dados disponibilizadas pelo [Mapa das Organizações da Sociedade Civil](https://mapaosc.ipea.gov.br/base-dados).

## Configurando o Ambiente

Para manutenção da simplicidade, os códigos necessários para realizar o processamento dos dados estão segmentados em módulos. Este caderno tem o propósito apenas de orquestrar e exibir os resultados dos processamentos realizados por tais módulos.

Para isto, é necessário reconfigurar o ambiente do notebook, para que os caminhos dos módulos sejam resolvidos corretamente, como em uma aplicação convencional Python.


In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")
print(f"Current working directory: {os.getcwd()}")

## Etapas do Processamento

Este caderno executa as seguintes etapas para produzir uma base de dados tratada:

### 1. Conversão de Encoding

Os datasets disponibilizados encontram-se em diversos formatos, além de contarem com "encodings" inadequados para a leitura e análise adequada através de algoritmos.

Para cada dataset original, esta etapa gera fontes contendo encoding adequado (em utf8) para a realização de etapas posteriores.

### 2. Tratamento de Organizações

#### 2.1. Tratamento de Áreas de Atuação

### 3. Tratamento de Endereços

### 4. Tratamento de Projetos


In [ ]:
import pandas
from data.processing.data_parser import to_utf8

ONGS_DATASET = "base_2025_2.csv"

result_path = to_utf8(ONGS_DATASET)

if result_path is None:
    raise FileNotFoundError(
        f"File {ONGS_DATASET} not found or could not be converted to UTF-8.")

result_df = pandas.read_csv(result_path, sep=";", encoding="utf-8")
result_df.info()